# Solutions: Graphs & Dependency Resolution

## Solution 1: Task Scheduler

In [ ]:
from typing import List, Dict, Set, Optional
from collections import defaultdict, deque
from dataclasses import dataclass

@dataclass
class Task:
    name: str
    depends_on: List[str]
    duration: int


class TaskScheduler:
    """Schedule tasks in dependency order."""
    
    def __init__(self, tasks: List[Task]):
        self.tasks = {task.name: task for task in tasks}
        self.adjacency_list: Dict[str, List[str]] = defaultdict(list)
        
        self._build_graph()
        self._validate()
    
    def _build_graph(self) -> None:
        for task in self.tasks.values():
            for dep in task.depends_on:
                self.adjacency_list[dep].append(task.name)
    
    def _validate(self) -> None:
        # Check missing dependencies
        for task in self.tasks.values():
            for dep in task.depends_on:
                if dep not in self.tasks:
                    raise ValueError(f"Task '{task.name}' depends on '{dep}' which doesn't exist")
        
        # Check cycles
        visited = set()
        rec_stack = set()
        
        def visit(name: str, path: List[str]) -> Optional[List[str]]:
            if name in rec_stack:
                cycle_start = path.index(name)
                return path[cycle_start:] + [name]
            if name in visited:
                return None
            
            visited.add(name)
            rec_stack.add(name)
            path.append(name)
            
            for dependent in self.adjacency_list[name]:
                cycle = visit(dependent, path[:])
                if cycle:
                    return cycle
            
            rec_stack.remove(name)
            return None
        
        for task_name in self.tasks.keys():
            if task_name not in visited:
                cycle = visit(task_name, [])
                if cycle:
                    raise ValueError(f"Circular dependency: {' -> '.join(cycle)}")
    
    def get_execution_order(self) -> List[str]:
        """Topological sort."""
        in_degree = {name: len(task.depends_on) for name, task in self.tasks.items()}
        queue = deque([name for name, degree in in_degree.items() if degree == 0])
        result = []
        
        while queue:
            task_name = queue.popleft()
            result.append(task_name)
            
            for dependent in self.adjacency_list[task_name]:
                in_degree[dependent] -= 1
                if in_degree[dependent] == 0:
                    queue.append(dependent)
        
        return result
    
    def get_parallel_batches(self) -> List[List[str]]:
        """Execution layers."""
        in_degree = {name: len(task.depends_on) for name, task in self.tasks.items()}
        batches = []
        remaining = set(self.tasks.keys())
        
        while remaining:
            current_batch = [name for name in remaining if in_degree[name] == 0]
            batches.append(current_batch)
            
            for task_name in current_batch:
                remaining.remove(task_name)
                for dependent in self.adjacency_list[task_name]:
                    if dependent in remaining:
                        in_degree[dependent] -= 1
        
        return batches
    
    def estimate_completion_time(self) -> int:
        """Sum of max duration per batch."""
        batches = self.get_parallel_batches()
        total_time = 0
        
        for batch in batches:
            batch_duration = max(self.tasks[name].duration for name in batch)
            total_time += batch_duration
        
        return total_time


# Test
tasks = [
    Task("design", [], 60),
    Task("frontend", ["design"], 120),
    Task("backend", ["design"], 180),
    Task("database", ["design"], 90),
    Task("integration", ["frontend", "backend", "database"], 60),
    Task("testing", ["integration"], 120),
]

scheduler = TaskScheduler(tasks)
print("Execution order:", scheduler.get_execution_order())
print("\nParallel batches:")
for i, batch in enumerate(scheduler.get_parallel_batches(), 1):
    print(f"  Batch {i}: {batch}")
print(f"\nEstimated completion time: {scheduler.estimate_completion_time()} minutes")

## Solution 2: Detect All Cycles

In [ ]:
def find_all_cycles(adjacency_list: Dict[str, List[str]]) -> List[List[str]]:
    """Find all cycles in a directed graph."""
    all_nodes = set(adjacency_list.keys())
    for neighbors in adjacency_list.values():
        all_nodes.update(neighbors)
    
    cycles = []
    visited = set()
    
    def visit(node: str, rec_stack: Set[str], path: List[str]):
        if node in rec_stack:
            # Found a cycle
            cycle_start = path.index(node)
            cycle = path[cycle_start:] + [node]
            # Normalize cycle (start from min node to avoid duplicates)
            min_idx = cycle.index(min(cycle))
            normalized = cycle[min_idx:-1] + cycle[:min_idx] + [cycle[min_idx]]
            if normalized not in cycles:
                cycles.append(normalized)
            return
        
        if node in visited:
            return
        
        visited.add(node)
        rec_stack.add(node)
        path.append(node)
        
        for neighbor in adjacency_list.get(node, []):
            visit(neighbor, rec_stack, path[:])
        
        rec_stack.remove(node)
    
    for node in all_nodes:
        visit(node, set(), [])
    
    return cycles


# Test
graph = {
    "A": ["B"],
    "B": ["C"],
    "C": ["A"],
    "D": ["E"],
    "E": ["D"],
}

cycles = find_all_cycles(graph)
print("Cycles found:")
for cycle in cycles:
    print(f"  {' -> '.join(cycle)}")

## Solution 3: Course Prerequisites

In [ ]:
def course_order(num_courses: int, prerequisites: List[tuple[int, int]]) -> List[int] | None:
    """Find valid course order using topological sort."""
    # Build adjacency list
    adjacency_list = defaultdict(list)
    in_degree = {i: 0 for i in range(num_courses)}
    
    for course, prereq in prerequisites:
        adjacency_list[prereq].append(course)
        in_degree[course] += 1
    
    # Kahn's algorithm
    queue = deque([course for course, degree in in_degree.items() if degree == 0])
    result = []
    
    while queue:
        course = queue.popleft()
        result.append(course)
        
        for dependent in adjacency_list[course]:
            in_degree[dependent] -= 1
            if in_degree[dependent] == 0:
                queue.append(dependent)
    
    # Check if all courses processed
    if len(result) != num_courses:
        return None  # Cycle detected
    
    return result


# Test 1: Valid
result = course_order(4, [(1, 0), (2, 0), (3, 1), (3, 2)])
print("Course order:", result)

# Test 2: Cycle
result = course_order(2, [(0, 1), (1, 0)])
print("Course order with cycle:", result)

## Solution 4: Transitive Dependencies

In [ ]:
def get_all_dependencies(
    package: str,
    dependencies: Dict[str, List[str]]
) -> Set[str]:
    """BFS to collect all transitive dependencies."""
    all_deps = set()
    queue = deque([package])
    visited = {package}
    
    while queue:
        current = queue.popleft()
        
        for dep in dependencies.get(current, []):
            all_deps.add(dep)
            if dep not in visited:
                visited.add(dep)
                queue.append(dep)
    
    return all_deps


# Test
deps = {
    "A": ["B", "C"],
    "B": ["D"],
    "C": ["D", "E"],
    "D": [],
    "E": ["F"],
    "F": [],
}

all_deps = get_all_dependencies("A", deps)
print("All dependencies of A:", sorted(all_deps))

## Solution 5: Build Order with Circular Detection

In [ ]:
def build_order(
    projects: Dict[str, List[str]]
) -> tuple[List[str] | None, List[str] | None]:
    """Determine build order or find circular dependency."""
    # Build adjacency list
    adjacency_list = defaultdict(list)
    all_projects = set(projects.keys())
    in_degree = {proj: 0 for proj in all_projects}
    
    for proj, deps in projects.items():
        for dep in deps:
            adjacency_list[dep].append(proj)
            in_degree[proj] += 1
            all_projects.add(dep)
    
    # Add projects with no dependencies
    for proj in all_projects:
        if proj not in in_degree:
            in_degree[proj] = 0
    
    # Try topological sort
    queue = deque([proj for proj, degree in in_degree.items() if degree == 0])
    result = []
    
    while queue:
        proj = queue.popleft()
        result.append(proj)
        
        for dependent in adjacency_list[proj]:
            in_degree[dependent] -= 1
            if in_degree[dependent] == 0:
                queue.append(dependent)
    
    # Check if successful
    if len(result) == len(all_projects):
        return result, None
    
    # Find cycle using DFS
    visited = set()
    rec_stack = set()
    
    def find_cycle(proj: str, path: List[str]) -> List[str] | None:
        if proj in rec_stack:
            cycle_start = path.index(proj)
            return path[cycle_start:] + [proj]
        if proj in visited:
            return None
        
        visited.add(proj)
        rec_stack.add(proj)
        path.append(proj)
        
        for dependent in adjacency_list[proj]:
            cycle = find_cycle(dependent, path[:])
            if cycle:
                return cycle
        
        rec_stack.remove(proj)
        return None
    
    for proj in all_projects:
        if proj not in visited:
            cycle = find_cycle(proj, [])
            if cycle:
                return None, cycle
    
    return None, None


# Test 1: Valid
projects = {
    "app": ["utils", "data"],
    "utils": [],
    "data": ["utils"],
    "api": ["data"],
}
order, cycle = build_order(projects)
print("Build order:", order)
print("Cycle:", cycle)

# Test 2: Cycle
projects_cyclic = {
    "app": ["api"],
    "api": ["data"],
    "data": ["app"],
}
order, cycle = build_order(projects_cyclic)
print("\nBuild order:", order)
print("Cycle:", cycle)

## Bonus Solution: Minimum Build Time

In [ ]:
def minimum_build_time(
    tasks: Dict[str, tuple[List[str], int]]
) -> tuple[int, Dict[str, int]]:
    """Calculate minimum build time with parallel execution."""
    # Build adjacency list and calculate in-degrees
    adjacency_list = defaultdict(list)
    in_degree = {name: 0 for name in tasks.keys()}
    
    for task, (deps, _) in tasks.items():
        in_degree[task] = len(deps)
        for dep in deps:
            adjacency_list[dep].append(task)
    
    # Topological sort with finish time calculation
    queue = deque([task for task, degree in in_degree.items() if degree == 0])
    finish_times = {}
    
    while queue:
        task = queue.popleft()
        deps, duration = tasks[task]
        
        # Start time = max finish time of dependencies
        start_time = max((finish_times[dep] for dep in deps), default=0)
        finish_times[task] = start_time + duration
        
        # Process dependents
        for dependent in adjacency_list[task]:
            in_degree[dependent] -= 1
            if in_degree[dependent] == 0:
                queue.append(dependent)
    
    total_time = max(finish_times.values())
    return total_time, finish_times


# Test
tasks = {
    "A": ([], 3),
    "B": (["A"], 2),
    "C": (["A"], 4),
    "D": (["B", "C"], 1),
}

total, finish_times = minimum_build_time(tasks)
print("Minimum build time:", total)
print("Finish times:", finish_times)

# Verify
assert finish_times["A"] == 3, "A finishes at 3"
assert finish_times["B"] == 5, "B starts at 3, finishes at 5"
assert finish_times["C"] == 7, "C starts at 3, finishes at 7"
assert finish_times["D"] == 8, "D starts at 7 (max of B=5, C=7), finishes at 8"
assert total == 8
print("\n✓ All assertions passed")

## 🎓 Key Patterns

1. **Topological Sort = Kahn's Algorithm**
   - Queue nodes with in-degree 0
   - Process, reduce neighbors' in-degree
   - If all processed → valid, else cycle

2. **Cycle Detection = DFS with Recursion Stack**
   - `visited`: Fully processed nodes
   - `rec_stack`: Current path
   - Back edge (node in `rec_stack`) = cycle

3. **Execution Layers = Iterative In-Degree**
   - Each layer = nodes with in-degree 0
   - Remove layer, recalculate in-degrees
   - Repeat until empty

4. **Transitive Closure = BFS/DFS**
   - Start from node
   - Traverse all reachable nodes
   - Collect visited set

5. **Critical Path = Topological + Max Accumulation**
   - Process nodes in topological order
   - Finish time = max(dep finish times) + duration
   - Total = max finish time